# A short & practical introduction to Tensor Flow!

Part 4

The goal of this notebook is to train a LSTM character prediction model over [Text8](http://mattmahoney.net/dc/textdata) data.

This is a personal wrap-up of all the material provided by [Google's Deep Learning course on Udacity](https://www.udacity.com/course/deep-learning--ud730), so all credit goes to them. 

Author: Pablo M. Olmos (olmos@tsc.uc3m.es)

** This notebook cerntainly needs more comments to make it self-contained**

Date: March 2017

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
# Lets check what version of tensorflow we have installed. The provided scripts should run with tf 1.0 and above

print(tf.__version__)

1.3.0


In [4]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename


filename = maybe_download('../../DataSets/textWordEmbeddings/text8.zip', 31344016) ## Change according to the folder where you saved the dataset provided

Found and verified ../../DataSets/textWordEmbeddings/text8.zip


In [5]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        name = f.namelist()[0]
        data = tf.compat.as_str(f.read(name))
    return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [6]:
text[0:20]

' anarchism originate'

Create a small validation set

In [7]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back

In [8]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0

In [9]:
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '
    
print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [10]:
batch_size=64      ## Number of batches, but also number of segments in which we divide the text. We read batch_size 
                   ## batches in parallel, each read from a different segment. The implementation is not obvious, the
                   ## key seems to be the zip function inside the for loop below
        
num_unrollings=10  ## Each sequence is num_unrolling character long

### NOW I GET IT!! Every batch is a batch_size times 27 (num letters) matrix. Every row correspond to a letter. Each letter 
### comes from a different sequence of (num_unrollings) so that the 64 letters cannot be read together.
## In the next batch, we have the following letter for each of the 64 training sequences!!

class BatchGenerator(object):
    
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size #We split the text into batch_size pieces
        self._cursor = [ offset * segment for offset in range(batch_size)]    #Cursor pointing every piece
        self._last_batch = self._next_batch()
  
    #
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0     #One hot encoding
            #print(self._text[self._cursor[b]])
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches
    
    
def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (mostl likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]            #Clever! The ZIP is the key function here!
    return s    

train_batches = BatchGenerator(train_text, batch_size, 10)
valid_batches = BatchGenerator(valid_text, 1, 1)



In [11]:
print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [12]:
#OK with this one
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

#OK with this one
def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
        probabilities.
    """
        
    r = random.uniform(0,1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

#OK with this one
def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p


def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b / np.sum(b, 1)[:, None]


In [13]:
train_batches.next()[0].shape

(64, 27)

# Simple LSTM Model

Recall the fundamental model


<img src="files/figLSTM.png">

Also, the un-regularized cost function is

\begin{align}
J(\boldsymbol{\theta})=\frac{1}{N}\sum_{n=1}^N\sum_{t=1}^{T_n}d(\boldsymbol{y}_t^{(n)},\sigma(\boldsymbol{h}_t^{(n)}))
\end{align}
where $d(\cdot,\cdot)$ is the cross-entropy loss function.

About the TF implementation below, see the following excellent [post](http://www.thushv.com/sequential_modelling/long-short-term-memory-lstm-networks-implementing-with-tensorflow-part-2/)

> 
Now calculating logits for softmax is a little bit tricky. This a temporal (time-based) network. So after each processing each num_unrolling batches through the LSTM cell, we update h_{t-1}=h_t and c_{t-1}=c_t before calculating logits and the loss. This is done by using tf.control_dependencies. What this does is that, logits will not be calculated until saved_output and saved_states are updated. Finally, as you can see, num_unrolling acts as the amount of history we are remembering.

In other words, in the computation graph everytime something is updated, all the dependent op nodes are updated and this is propagated through the graph. If we want to wait until the very end to compute the loss, we wait using the command tf.control_dependencies.

About the zip() and zip(*) operators, see this [post](https://docs.python.org/2/library/functions.html#zip)

In [14]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
    # Parameters:
    
    #i(t) parameters
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))   ##W^ix
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1)) ## W^ih
    ib = tf.Variable(tf.zeros([1, num_nodes])) ##b_i
    
    #f(t) parameters
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1)) ##W^fx
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1)) ##W^fh
    fb = tf.Variable(tf.zeros([1, num_nodes])) ##b_f
    
    #g(t) parameters
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1)) ##W^gx
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1)) ##W^gh
    cb = tf.Variable(tf.zeros([1, num_nodes]))  ##b_g
    
    #o(t) parameters
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))  ##W^ox
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))  ##W^oh
    ob = tf.Variable(tf.zeros([1, num_nodes])) ##b_o
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False) #h(t)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False) #s(t)
    
    
    # Classifier weights and biases (over h(t) to labels)
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb       
        state = forget_gate * state + input_gate * tf.tanh(update)    #tf.tanh(update) is g(t)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state      #h(t) is output_gate * tf.tanh(state)

    # Input data. Now it makes sense!!!
    
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    
    outputs = list()
    output = saved_output
    aux = output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),saved_state.assign(state)]):
        #Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(axis=0,values=outputs), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(axis=0, values=train_labels),logits=logits))

    # Optimizer.
    
    """Next, we are implementing the optimizer. Remember! we should use “gradient clipping” (tf.clip_by_global_norm) 
    to avoid “Exploding gradient” phenomenon. Also, we decay the learning_rate over time."""
    global_step = tf.Variable(0)
    
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    """ optimizer.compute_gradients(loss) yields (gradient, value) tuples. gradients, v = zip(*optimizer.compute_gradients(loss))
    performs a transposition, creating a list of gradients and a list of values.
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    then clips the gradients, and optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    re-zips the gradient and value lists back into an iterable of (gradient, value) 
    tuples which is then passed to the optimizer.apply_gradients method."""
    
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
  
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    # Create an op that groups multiple operations.
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])),
                                  saved_sample_state.assign(tf.zeros([1, num_nodes])))
    
    sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
    
    with tf.control_dependencies([saved_sample_output.assign(sample_output),saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


In [15]:
num_steps = 1001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run(
            [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss /= summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print(
                'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(
                valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.294209 learning rate: 10.000000
Minibatch perplexity: 26.96
mxovr oqde yfxkdo lgo slt r jyl  uqoizrtvtz rozih o  giopvwtirvvfji ioisixishdut
rteew vmekfznhts utnotduqtnvitnip niki eb e pv dbopsfvpm  el ittceecbnryravioe j
ltvbgee kzaauloap jdlvnh vthfvkvqtisnr oqnr yc  gs om icihhdfriiaradwnooezke v c
b zowgrzzlfsfin qtpnbleqteoiss pf tta jn wd ohsv  zgwsictzraulredxokf os ev qwab
lpeottcin ffinel hxcylubmceenohedvin cm nbp tednlatrxipat  yqvwnosbgnlwmviaeruox
Validation set perplexity: 20.10
Average loss at step 100: 2.598391 learning rate: 10.000000
Minibatch perplexity: 10.37
Validation set perplexity: 10.75
Average loss at step 200: 2.253659 learning rate: 10.000000
Minibatch perplexity: 9.72
Validation set perplexity: 9.11
Average loss at step 300: 2.097530 learning rate: 10.000000
Minibatch perplexity: 7.71
Validation set perplexity: 7.79
Average loss at step 400: 1.997222 learning rate: 10.000000
Minibatch perplexity: 7.69
Validation set per

In [16]:
batches = train_batches.next()

In [17]:
batches[0]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])